In [3]:
"""
RandLA-Net training for variable-sized pickle point clouds
Compatible with pc.pickle data structure
"""

import os
import pickle
import numpy as np
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# ==============================
# DATASET
# ==============================
class RandLANetDataset(Dataset):
    def __init__(self, data_folder):
        """
        Load point cloud pickles from subfolders: pc_id=*/pc.pickle
        """
        self.data = []

        for root, dirs, files in os.walk(data_folder):
            if 'pc.pickle' in files:
                pc_path = os.path.join(root, 'pc.pickle')
                md_path = os.path.join(root, 'metadata', 'metadata.pickle')
                try:
                    with open(pc_path, 'rb') as f:
                        pc = pickle.load(f)  # numpy array
                    md = None
                    if os.path.exists(md_path):
                        with open(md_path, 'rb') as f:
                            md = pickle.load(f)
                    self.data.append((pc, md))
                except Exception as e:
                    print(f"Failed to load {pc_path}: {e}")

        print(f"Total samples found: {len(self.data)}")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        pc, md = self.data[idx]
        # pc shape: (N,7)  -> use first 6 features as input, last column can be ignored
        coords_feats = pc[:, :6].astype(np.float32)  # xyz + intensity + others
        labels = pc[:, -1].astype(np.int64)          # last column as label

        return {
            "features": torch.tensor(coords_feats, dtype=torch.float32),
            "labels": torch.tensor(labels, dtype=torch.long)
        }

# ==============================
# COLLATE FUNCTION
# ==============================
def collate_fn(batch):
    return {
        "features": [b["features"] for b in batch],
        "labels": [b["labels"] for b in batch]
    }

# ==============================
# RANDLA-NET MODEL (simplified)
# ==============================
class SharedMLP(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv = nn.Conv1d(in_channels, out_channels, 1)
        self.bn = nn.BatchNorm1d(out_channels)
        self.relu = nn.ReLU(inplace=True)
    def forward(self, x):
        return self.relu(self.bn(self.conv(x)))

class RandlaNet(nn.Module):
    def __init__(self, d_in=6, d_out=[32,64,128], num_classes=4, dropout=0.5):
        super().__init__()
        self.encoder = nn.ModuleList()
        current_d = d_in
        for d in d_out:
            self.encoder.append(SharedMLP(current_d, d))
            current_d = d
        self.decoder = nn.ModuleList()
        for i in range(len(d_out)-1, 0, -1):
            self.decoder.append(SharedMLP(d_out[i]+d_out[i-1], d_out[i-1]))
        self.dropout = nn.Dropout(dropout)
        self.fc1 = nn.Conv1d(d_out[0], 64, 1)
        self.bn1 = nn.BatchNorm1d(64)
        self.fc2 = nn.Conv1d(64, num_classes, 1)

    def forward(self, x):
        # x: [B, N, d_in]
        x = x.transpose(1,2)  # [B, d_in, N]
        skips = []
        for layer in self.encoder:
            x = layer(x)
            skips.append(x)
        for i, layer in enumerate(self.decoder):
            skip = skips[-(i+2)]
            x = torch.cat([x, skip], dim=1)
            x = layer(x)
        x = self.dropout(x)
        x = torch.relu(self.bn1(self.fc1(x)))
        x = self.fc2(x)
        return x.transpose(1,2)  # [B,N,num_classes]

# ==============================
# FOCAL LOSS
# ==============================
class FocalLoss(nn.Module):
    def __init__(self, alpha=None, gamma=2.0):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
    def forward(self, inputs, targets):
        B,N,C = inputs.shape
        inputs = inputs.reshape(-1,C)
        targets = targets.reshape(-1)
        ce_loss = nn.functional.cross_entropy(inputs, targets, reduction='none')
        p = torch.exp(-ce_loss)
        loss = (1-p)**self.gamma * ce_loss
        if self.alpha is not None:
            alpha_t = self.alpha[targets]
            loss = alpha_t * loss
        return loss.mean()

# ==============================
# METRICS
# ==============================
def compute_metrics(preds, targets, num_classes=4):
    preds = preds.cpu().numpy()
    targets = targets.cpu().numpy()
    acc = 100*(preds==targets).sum()/targets.size
    ious = []
    for c in range(num_classes):
        pred_c = (preds==c)
        target_c = (targets==c)
        inter = (pred_c & target_c).sum()
        union = (pred_c | target_c).sum()
        ious.append(inter/union if union>0 else 1.0)
    return acc, np.mean(ious), ious

# ==============================
# TRAINING LOOP
# ==============================
def train_model(model, dataloader, epochs=5, device="cpu", num_classes=4, lr=0.001):
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = FocalLoss(alpha=torch.ones(num_classes).to(device), gamma=2.0)

    for epoch in range(epochs):
        print(f"\nEpoch {epoch+1}/{epochs}")
        total_loss = 0
        all_acc, all_miou = [], []
        class_ious_sum = np.zeros(num_classes)
        clouds_count = 0

        for batch in tqdm(dataloader):
            optimizer.zero_grad()
            for features, labels in zip(batch["features"], batch["labels"]):
                features = features.unsqueeze(0).to(device)  # [1,N,d_in]
                labels   = labels.unsqueeze(0).to(device)    # [1,N]
                outputs  = model(features)                    # [1,N,num_classes]
                loss     = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                total_loss += loss.item()
                preds = torch.argmax(outputs, dim=-1)        # [1,N]
                acc, miou, ious = compute_metrics(preds, labels, num_classes)
                all_acc.append(acc)
                all_miou.append(miou)
                class_ious_sum += np.array(ious)
                clouds_count += 1

        avg_acc = np.mean(all_acc)
        avg_miou = np.mean(all_miou)
        avg_class_ious = class_ious_sum / clouds_count
        print(f"Loss: {total_loss/len(dataloader):.4f} | Acc: {avg_acc:.2f}% | mIoU: {avg_miou:.4f}")
        print(f"IoU per class: {[f'{iou:.4f}' for iou in avg_class_ious]}")

# ==============================
# MAIN
# ==============================
if __name__ == "__main__":
    DATA_FOLDER = r"C:/Users/umair.muhammad/RandLA-Net-pytorch/data"
    dataset = RandLANetDataset(DATA_FOLDER)
    loader = DataLoader(dataset, batch_size=2, shuffle=True, collate_fn=collate_fn, num_workers=0)

    CLASSES = 4
    model = RandlaNet(d_in=6, d_out=[32,64,128], num_classes=CLASSES)

    train_model(model, loader, epochs=10, device="cpu", num_classes=CLASSES)


Total samples found: 15

Epoch 1/10


100%|██████████| 8/8 [00:11<00:00,  1.38s/it]


Loss: 0.8307 | Acc: 81.37% | mIoU: 0.2063
IoU per class: ['0.8147', '0.0056', '0.0046', '0.0004']

Epoch 2/10


100%|██████████| 8/8 [00:10<00:00,  1.37s/it]


Loss: 0.3171 | Acc: 97.22% | mIoU: 0.3432
IoU per class: ['0.9722', '0.0004', '0.0001', '0.4000']

Epoch 3/10


...

Loss: 0.1095 | Acc: 97.30% | mIoU: 0.5433
IoU per class: ['0.9730', '0.0003', '0.3333', '0.8667']

Epoch 9/10


100%|██████████| 8/8 [00:11<00:00,  1.45s/it]


Loss: 0.1087 | Acc: 97.30% | mIoU: 0.5933
IoU per class: ['0.9730', '0.0002', '0.5333', '0.8667']

Epoch 10/10


100%|██████████| 8/8 [00:11<00:00,  1.43s/it]

Loss: 0.1064 | Acc: 97.30% | mIoU: 0.5766
IoU per class: ['0.9730', '0.0003', '0.4667', '0.8667']


In [16]:
torch.save(model.state_dict(), "C:/Users/umair.muhammad/RandLA-Net-pytorch/data/best_randlanet.pth")


In [13]:
import os
print(os.getcwd())


C:\Users\umair.muhammad\RandLA-Net-pytorch\model


In [21]:
import os
import pickle
import torch
import numpy as np
import open3d as o3d
from randla_model import RandlaNet
from sklearn.neighbors import NearestNeighbors

# ------------------------------
DEVICE = "cpu"
NUM_CLASSES = 4
PC_FOLDER = r"C:/Users/umair.muhammad/RandLA-Net-pytorch/data/pc_id=11"
PC_FILE = os.path.join(PC_FOLDER, "pc.pickle")
MODEL_FILE = r"C:/Users/umair.muhammad/RandLA-Net-pytorch/model/best_randlanet.pth"
K = 16  # number of neighbors for RandLA-Net

# ------------------------------
# LOAD POINT CLOUD
pc_data = pickle.load(open(PC_FILE, "rb"))
xyz = pc_data[:, :3]        # use xyz only
N = xyz.shape[0]

# convert to torch
xyz_tensor = torch.tensor(xyz, dtype=torch.float32).unsqueeze(0).to(DEVICE)
features_tensor = torch.tensor(xyz, dtype=torch.float32).unsqueeze(0).to(DEVICE)  # xyz as features

# ------------------------------
# COMPUTE KNN indices
nbrs = NearestNeighbors(n_neighbors=K, algorithm='auto').fit(xyz)
distances, neigh_idx_np = nbrs.kneighbors(xyz)
neigh_idx = torch.tensor(neigh_idx_np, dtype=torch.long).unsqueeze(0).to(DEVICE)  # [1,N,K]

# batch placeholders
sub_idx = torch.arange(N).unsqueeze(0).to(DEVICE)
interp_idx = torch.arange(N).unsqueeze(0).to(DEVICE)

# ------------------------------
# LOAD MODEL
model = RandlaNet(d_out=[32,64,128], n_layers=3, n_classes=NUM_CLASSES)
model.load_state_dict(torch.load(MODEL_FILE, map_location=DEVICE))
model.to(DEVICE)
model.eval()

# ------------------------------
# INFERENCE
with torch.no_grad():
    inputs = {
        "features": [features_tensor.squeeze(0)],  # list of [N,d_in]
        "xyz": [xyz_tensor.squeeze(0)],            # list of [N,3]
        "neigh_idx": [neigh_idx.squeeze(0)],       # list of [N,K]
        "sub_idx": [sub_idx.squeeze(0)],           # list of [N]
        "interp_idx": [interp_idx.squeeze(0)]      # list of [N]
    }
    outputs = model(inputs)                        # [1,N,num_classes]
    preds = torch.argmax(outputs, dim=-1).squeeze(0).cpu().numpy()  # [N]

# ------------------------------
# VISUALIZATION
colors_map = [
    [1,0,0],   # class 0
    [0,1,0],   # class 1
    [0,0,1],   # class 2
    [1,1,0],   # class 3
]
colors = np.array([colors_map[c] for c in preds])

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz)
pcd.colors = o3d.utility.Vector3dVector(colors)
o3d.visualization.draw_geometries([pcd])


In [24]:
import os
import pickle
import torch
import numpy as np
import open3d as o3d
from randla_model import RandlaNet
from sklearn.neighbors import NearestNeighbors

# ------------------------------
# SETTINGS
# ------------------------------
DATA_FOLDER = r"C:/Users/umair.muhammad/RandLA-Net-pytorch/data/pc_id=15"
MODEL_FILE  = r"C:/Users/umair.muhammad/RandLA-Net-pytorch/model/best_randlanet.pth"
DEVICE = "cpu"
NUM_CLASSES = 4  # same as training
K = 16  # neighbor size for RandLA-Net

# ------------------------------
# LOAD DATA
# ------------------------------
pc_file = os.path.join(DATA_FOLDER, "pc.pickle")
with open(pc_file, "rb") as f:
    pc = pickle.load(f)

# Assuming your pc.pickle is (N,7) -> [x,y,z,feat1,feat2,feat3,label]
xyz = pc[:, :3]               # coordinates
features = pc[:, 3:6]         # same features as used in training

# Convert to torch tensors
xyz_tensor = torch.tensor(xyz, dtype=torch.float32).unsqueeze(0).to(DEVICE)       # [1,N,3]
features_tensor = torch.tensor(features, dtype=torch.float32).unsqueeze(0).to(DEVICE)  # [1,N,C]

# ------------------------------
# PREPARE NEIGHBORS
# ------------------------------
nbrs = NearestNeighbors(n_neighbors=K, algorithm='auto').fit(xyz)
distances, indices = nbrs.kneighbors(xyz)
indices_tensor = torch.tensor(indices, dtype=torch.long).unsqueeze(0).to(DEVICE)  # [1,N,K]

# ------------------------------
# LOAD MODEL
# ------------------------------
model = RandlaNet(d_out=[32,64,128], n_layers=3, n_classes=NUM_CLASSES)
model.load_state_dict(torch.load(MODEL_FILE, map_location=DEVICE))
model.to(DEVICE)
model.eval()

# ------------------------------
# INFERENCE
# ------------------------------
with torch.no_grad():
    inputs = {
        "features": [features_tensor.squeeze(0)],  # list of [N,C]
        "xyz": [xyz_tensor.squeeze(0)],            # list of [N,3]
        "neigh_idx": [indices_tensor.squeeze(0)],  # list of [N,K]
        "interp_idx": [torch.arange(xyz.shape[0], dtype=torch.long)]  # identity
    }
    outputs = model(inputs)  # [1,N,num_classes]
    preds = torch.argmax(outputs, dim=-1).squeeze(0).cpu().numpy()  # [N]

# ------------------------------
# VISUALIZATION
# ------------------------------
# Define distinct colors per class
colors = np.array([
    [1, 0, 0],   # red
    [0, 1, 0],   # green
    [0, 0, 1],   # blue
    [1, 1, 0],   # yellow
], dtype=np.float32)

# Assign colors
pc_colors = colors[preds]

# Create Open3D point cloud
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz)
pcd.colors = o3d.utility.Vector3dVector(pc_colors)

# Show
o3d.visualization.draw_geometries([pcd])
